# Predicting Wind Turbine Output with a Random Forest

In this analysis, the shape of the data frame is taken directly from the original data file: each record contains measurements from a single zone, even though the power output from the wind turbine in that zone is correlated with wind velocity measurements in other zones.

1. [Imports.](#Cell1)
1. [Data preprocessing function for training and testing data sets.](#Cell2)
1. [Preprocess training data set and split it into training and testing subsets.](#Cell3)
1. [Fit random forest model to training subset of training data set.](#Cell4)
1. [Do a grid search to find optimal hyper-parameters of the random forest regressor.](#Cell5)
1. [Evaluate fit results.](#Cell6)
1. [Fit random forest model to entire training data set.](#Cell7)
1. [Prepare test data set in the same way as the training set, make predictions, and create output csv file.](#Cell8)

<a id='Cell1'></a>

In [1]:
'''
Imports
'''
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
import csv
from time import strftime

<a id='Cell2'></a>

In [2]:
'''
Data preprocessing function for training and testing data sets.
'''

def PreProcess(fname):
    '''
    Read in wind turbine data and store them in a Pandas dataframe after cleanup.
    The input csv file fname contains the following fields:

    ID:         Unique ID for each observation
    ZONEID:     Zone/Turbine ID, a number between 1 and 10
    TIMESTAMP:  Time of observation, in the format "YYYYMMDD h:mm" or "YYYYMMDD hh:mm"
    TARGETVAR:  Wind turbine output
    U10:        Zonal Wind Vector at 10 m
    V10:        Meridional Wind Vector at 10 m
    U100:       Zonal Wind vector at 100 m
    V100:       Meridional Wind vector at 100 m
    '''

    # Read in the windmill data, parsing the third column as datetimes.
    Train            = "Train" in fname
    df0              = pd.read_csv(fname, header=0, parse_dates=[2])

    # Extract year, day of year, and hour from timestamp, then drop timestamp column
    df0["Year"]      = df0["TIMESTAMP"].map(lambda x: x.year)
    df0["DayOfYear"] = df0["TIMESTAMP"].map(lambda x: x.timetuple().tm_yday)
    df0["Hour"]      = df0["TIMESTAMP"].map(lambda x: x.hour)
    df0              = df0.drop('TIMESTAMP', axis=1)

    # Rename columns
    if Train:
        df0.rename(columns={"ID": "Id", "ZONEID": "ZoneId", "TARGETVAR": "Target"}, inplace=True)
    else:
        df0.rename(columns={"ID": "Id", "ZONEID": "ZoneId"}, inplace=True)

    # Convert categorical variables in dataframe.
    df0['ZoneId']    = df0['ZoneId'].astype(int).astype("category")
    df0              = pd.get_dummies(df0, drop_first=True)

    print('PreProcess: Dataframe Shape = {0}'.format(df0.shape))
    
    return df0

<a id='Cell3'></a>

In [3]:
'''
Preprocess training data set and split it into training and testing subsets.
'''

fname = "data/Train_O4UPEyW.csv"
df0   = PreProcess(fname)

# Convert pandas dataframe to its numpy representation, separating out features from target variable.
features  = df0.columns.values.tolist()
features.remove("Target")
features.remove("Id")
X         = df0.as_matrix(columns=features)
y         = np.array(df0["Target"].tolist())

# Create training and test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print('Size of data set for training: {0}, for testing: {1}\n'.format(len(y_train), len(y_test)))

PreProcess: Dataframe Shape = (138710, 18)
Size of data set for training: 97097, for testing: 41613



<a id='Cell4'></a>

In [4]:
'''
Fit random forest model to training subset of training data set.
'''

rfr_train = RandomForestRegressor(n_estimators=1000, criterion="mse", max_depth=30, min_samples_split=2, 
                                  min_samples_leaf=5, max_features="sqrt", max_leaf_nodes=None, bootstrap=True,
                                  oob_score=False, n_jobs=-1, random_state=0, verbose=0)
%time rfr_train.fit(X_train, y_train)

CPU times: user 5min 43s, sys: 4.28 s, total: 5min 47s
Wall time: 1min 44s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=5,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=-1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

<a id='Cell5'></a>

In [5]:
'''
Do a grid search to find optimal hyper-parameters of the random forest regressor.
Use Root Mean Squared Error (RMSE) to score parameter choices.

(This cell can be omitted if the previous one was run.)
'''

# Specify the parameter grid to be searched
parameter_grid = [{'n_estimators': [1000, 2000, 3000], 
                   'max_depth': [30, 40, 50], 
                   'min_samples_leaf': [3, 5, 7]}]

# Start the grid search
rfo = GridSearchCV(RandomForestRegressor(criterion='mse', min_samples_split=2, max_features="sqrt",
                                         max_leaf_nodes=None, bootstrap=True, oob_score=False, 
                                         n_jobs=-1, random_state=0, verbose=0),
                   parameter_grid, refit=True, cv=5, scoring='neg_mean_squared_error')
%time rfo.fit(X_train, y_train)

# Print out the results (flipping the sign on the grid scores to make them positive again).
print("Best parameter values found on development set:")
print(" ")
print(rfo.best_params_)
print(" ")
print("Grid scores on development set:")
print(" ")
for mean_score, std_score, settings in zip(rfo.cv_results_['mean_test_score'], 
                                           rfo.cv_results_['std_test_score'],
                                           rfo.cv_results_['params']):
    print("%0.5f (+/-%0.05f) for %r" %(-mean_score, 2*std_score, settings))

# Retrieve best random forest model from grid search
rfr_train = rfo.best_estimator_
print("\nBest estimator:\n%s" %rfr_train)

fName = 'rfr_train_' + strftime("%Y_%m_%d_%H_%M_%S")
path_to_file = 'fitted_models/'+fName+'.model'
joblib.dump(rfr_train, path_to_file)
print('\nRandom forest regression model saved to {0}'.format(path_to_file))

CPU times: user 21h 26min 45s, sys: 18min 34s, total: 21h 45min 20s
Wall time: 5h 58min 48s
Best parameter values found on development set:
 
{'n_estimators': 2000, 'max_depth': 40, 'min_samples_leaf': 3}
 
Grid scores on development set:
 
0.02116 (+/-0.00064) for {'n_estimators': 1000, 'max_depth': 30, 'min_samples_leaf': 3}
0.02115 (+/-0.00065) for {'n_estimators': 2000, 'max_depth': 30, 'min_samples_leaf': 3}
0.02115 (+/-0.00064) for {'n_estimators': 3000, 'max_depth': 30, 'min_samples_leaf': 3}
0.02199 (+/-0.00066) for {'n_estimators': 1000, 'max_depth': 30, 'min_samples_leaf': 5}
0.02198 (+/-0.00066) for {'n_estimators': 2000, 'max_depth': 30, 'min_samples_leaf': 5}
0.02198 (+/-0.00066) for {'n_estimators': 3000, 'max_depth': 30, 'min_samples_leaf': 5}
0.02259 (+/-0.00068) for {'n_estimators': 1000, 'max_depth': 30, 'min_samples_leaf': 7}
0.02259 (+/-0.00069) for {'n_estimators': 2000, 'max_depth': 30, 'min_samples_leaf': 7}
0.02259 (+/-0.00069) for {'n_estimators': 3000, 'max_de

<a id='Cell6'></a>

In [5]:
'''
Evaluate fit results.
'''
FeatureImportances = [[feature, importance] for (feature,importance) in zip(features,rfr_train.feature_importances_)]
print('Feature Importances:\n')
for fi in sorted(FeatureImportances, key=lambda x: x[1], reverse=True):
    print('   %10s: %f' %(fi[0],fi[1]))

y_train_pred = rfr_train.predict(X_train)
y_test_pred  = rfr_train.predict(X_test)
rmse_train   = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test    = np.sqrt(mean_squared_error(y_test, y_test_pred))

print('\nRMSE on train data set: {0}'.format(rmse_train))
print('RMSE on test data set:  {0}'.format(rmse_test))

Feature Importances:

         V100: 0.249530
         U100: 0.229922
          V10: 0.183493
          U10: 0.173526
    DayOfYear: 0.040154
         Hour: 0.036495
    ZoneId_10: 0.019304
     ZoneId_3: 0.012043
     ZoneId_6: 0.010053
     ZoneId_5: 0.009071
     ZoneId_4: 0.007283
     ZoneId_8: 0.006840
     ZoneId_7: 0.006761
     ZoneId_2: 0.006053
         Year: 0.004861
     ZoneId_9: 0.004610

RMSE on train data set: 0.112916130391
RMSE on test data set:  0.14669382404


<a id='Cell7'></a>

In [6]:
'''
Fit random forest model to entire training data set.
'''
settings = rfr_train.get_params()
rfr      = RandomForestRegressor(**settings)
%time rfr.fit(X, y)

CPU times: user 6min 59s, sys: 4.39 s, total: 7min 3s
Wall time: 1min 59s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=5,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=-1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

<a id='Cell8'></a>

In [7]:
'''
Prepare test data set in the same way as the training set, make predictions, and create output csv file.
'''

# Preprocess testing data set.
fname       = "data/Test_uP7dymh.csv"
ttdf0       = PreProcess(fname)

# Save Id column to list.
ttid        = ttdf0['Id'].tolist()

# Count how many observations we have in August and September 2013
nTotal      = len(ttdf0)
nAugSep2013 = len([DoY for DoY in ttdf0['DayOfYear'] if DoY>212 and DoY<274])
print('\nTotal number of observation times in test set:       {0}'.format(nTotal))
print('Number of observations in August and September 2013: {0}'.format(nAugSep2013))

# Compute predictions for test set.
ttX         = ttdf0.as_matrix(columns=features)
tty         = rfr.predict(ttX)

# Generate output csv file
fName_out   = 'data/rfr_' + strftime("%Y_%m_%d_%H_%M_%S") + '.out.csv'
with open(fName_out, 'w') as csvfile:
    writer  = csv.writer(csvfile)
    writer.writerow(["ID", "TARGETVAR"])
    for idval,yval in zip(ttid,tty):
        writer.writerow([idval,yval])
print('\nOutput written to {0}'.format(fName_out))

PreProcess: Dataframe Shape = (29290, 17)

Total number of observation times in test set:       29290
Number of observations in August and September 2013: 14640

Output written to data/rfr_2017_04_08_21_31_35.out.csv
